In [5]:
import pandas as pd
import os

In [7]:
#creating data part

In [41]:
images=[]
globalpath='C:\\Users\\jerar\\OneDrive\\Desktop\\food_classify\\images\\'
os.chdir(globalpath)
print(os.getcwd())
labels=os.listdir()#list of all foodnames since food name is the directory name


C:\Users\jerar\OneDrive\Desktop\food_classify\images


In [ ]:
### creating a data frame consisiting of following fields imagepath,label
###

In [85]:
data={"image":[],
    "label":[]
}
for label in labels:
    os.chdir(globalpath)
    path=os.getcwd()+"\\"+label
    os.chdir(path)
    imagelist=os.listdir()
    labellist=[]
    for i in range(0,len(imagelist)):
        imagelist[i]=path+"\\"+imagelist[i]
        labellist.append(label)
    data["image"].extend(imagelist)
    data["label"].extend(labellist)

#load data into a DataFrame object:
df = pd.DataFrame(data)

        

C:\Users\jerar\OneDrive\Desktop\food_classify\images\Biryani\00000000_resized.png
